# Updating Hessians
The most common Hessian update formula is the BFGS update.  This formula results in a positive-definite Hessian matrix with all real eigenvalues.  So it is not suitable for transition-state optimizations.  For the formulae for BFGS, Powell, MS, and other update schemes see J.M. Bofill, _J. Comp. Chem._, 15, 1 (1994), or V. Bakken and T. Helgaker, _J. Chem. Phys._, 117, 9160 (2002).

Below, we will take a geometry optimization step, to obtain internal coordinate values and forces at two different points on the potential energy surface.  We will use the BFGS update to update the Hessian matrix.  For details of how the actual step is taken, see the later tutorials.

In [1]:
import psi4
from psi4 import *
from psi4.core import *
import numpy as np

mol = psi4.geometry("""
O
H 1 0.9
H 1 0.9 2 104
""")
psi4.set_options({"basis": "cc-pvdz"})
mol.update_geometry()

# Make the internal coordinates manually.
from optking import stre, bend
intcos = [stre.STRE(0,1), stre.STRE(0,2), bend.BEND(1,0,2)]
for intco in intcos: 
    print (intco) 
Natom = 3
Nintco = len(intcos)

 R(1,2)
 R(1,3)
 B(2,1,3)


In [2]:
# Initialize optking
from optking import printInit
printInit()
import optking.optParams as op
userOptions={} # assume no special options
op.Params = op.OPT_PARAMS(userOptions)
mol = core.get_active_molecule()
import optking.molsys
Molsys = optking.molsys.MOLSYS.fromPsi4Molecule(mol)
Molsys._fragments[0].intcos = intcos # assign internals manually
xyz = np.array(mol.geometry())
Z = [int(mol.Z(i)) for i in range(Natom)]

H = np.zeros((Nintco,Nintco), float)
for i,intco in enumerate(intcos):
    H[i,i] = intco.diagonalHessianGuess(xyz, Z, guessType="SCHLEGEL")
print "Schlegel Guess Hessian for Water (in au)"
print (H)

# Compute internal coordinate forces
import math
def symmMatInv(A):
    dim = A.shape[0]
    det = 1.0

    evals, evects = np.linalg.eigh(A)
    evects = evects.T
    for i in range(dim):
        det *= evals[i]

    diagInv = np.zeros( (dim,dim), float)
    for i in range(dim):
        if math.fabs(evals[i]) > 1.0e-10:
            diagInv[i,i] = 1.0/evals[i]
            
    # A^-1 = P^t D^-1 P
    tmpMat = np.dot(diagInv, evects)
    AInv = np.dot(evects.T, tmpMat)
    return AInv

Ncart = 3*Natom

# Compute B and A^T matrices
from optking import intcosMisc
B = intcosMisc.Bmat(intcos, xyz)
G = np.dot(B, B.T)
Ginv = symmMatInv(G)
Atranspose = np.dot(Ginv, B)

q_1 = intcosMisc.qValues(intcos, Molsys.geom)
E_1 = psi4.energy('scf')
g_x1 = np.reshape( np.array( psi4.gradient('scf')), (3*Natom))

# Save initial step info
from optking import history
f_q1 = -1 * np.dot( Atranspose, g_x1)
history.History.append(Molsys.geom, E_1, f_q1); # Save initial step info.

from optking import stepAlgorithms
Dq = stepAlgorithms.Dq_NR(Molsys, E_1, f_q1, H)

# Put new geometry into place and compute gradient again
psi_geom = core.Matrix.from_array( Molsys.geom )
mol.set_geometry( psi_geom )
mol.update_geometry()

q_2 = intcosMisc.qValues(intcos, Molsys.geom)
g_x2 = np.reshape( np.array( psi4.gradient('scf')), (3*Natom))
f_q2 = -1 * np.dot(Atranspose, g_x2)

print "Internal Values first step"
print (q_1)
print "Internal Values second step"
print (q_2)

print "Internal Forces first step"
print (f_q1)
print "Internal Forces second step"
print (f_q2)


	Generating molecular system for optimization from PSI4.
	1 Fragments in PSI4 molecule object.
	Creating fragment 1 with 3 atoms
Schlegel Guess Hessian for Water (in au)
[[ 0.70672641  0.          0.        ]
 [ 0.          0.70672641  0.        ]
 [ 0.          0.          0.16      ]]
	Taking NR optimization step.
	Norm of target step-size    0.1366991994
	Projected energy change by quadratic approximation:        -0.0059734909
	Beginnning displacement in cartesian coordinates...
	Successfully converged to displaced geometry.

	       --- Internal Coordinate Step in ANG or DEG, aJ/ANG or AJ/DEG ---
	-----------------------------------------------------------------------------
	         Coordinate      Previous         Force        Change          New 
	         ----------      --------        ------        ------        ------
	             R(1,2)       0.90000       0.52698       0.04789       0.94789
	             R(1,3)       0.90000       0.52698       0.04789       0.94789
	   

Here we will carry out the BFGS update for the first two gradients.  The update may be carried out using all past gradients if desired, though information from distant parts of the potential surface may not be relevant or helpful.

In [3]:
H_new = np.zeros( (Nintco,Nintco) , float)
dq = np.zeros(Nintco, float)
dg = np.zeros(Nintco, float)

dq[:] = q_2 - q_1
dg[:] = f_q1 - f_q2   # gradients -- not forces!

gq = np.dot(dq, dg)

for i in range(Nintco):
    for j in range(Nintco):
        H_new[i,j] = H[i,j] + dg[i] * dg[j] / gq

Hdq = np.dot(H, dq)
dqHdq = np.dot(dq, Hdq)

for i in range(Nintco):
    for j in range(Nintco):
        H_new[i,j] -=  Hdq[i] * Hdq[j] / dqHdq
        
print "Updated Hessian with BFGS"
print (H_new)

Updated Hessian with BFGS
[[ 0.7137803   0.00705389  0.03334153]
 [ 0.00705389  0.7137803   0.03334153]
 [ 0.03334153  0.03334153  0.17092498]]


We see that the Hessian in this case was not much changed by the update.  The first diagonal (stretching) element changes from 0.707 -> 0.714, while the bend changes from 0.16 -> 0.17.

However, if one repeats this demonstration with an initial Hessian guess set to "SIMPLE", then the initial Hessian guess is poorer (0.50 for the stretch), and the changes made by the first update (e.g., to 0.59 for the stretch) are more substantial.